# Retrieval API DEMO

## Code Initialisation

In [15]:
import os
import re
import requests as r
from IPython.display import Markdown, HTML
import utils as u
from dotenv import load_dotenv

load_dotenv()

True

## Constants

In [16]:
API_HOST = 'api.dowjones.com'
AUTH_HOST = 'accounts.dowjones.com'
CLIENT_ID = os.getenv('FACTIVA_CLIENTID')
USERNAME = os.getenv('FACTIVA_USERNAME')
PASSWORD = os.getenv('FACTIVA_PASSWORD')
AUTH_URL = f"https://{AUTH_HOST}/oauth2/v1/token"
LLM_CONTEXT_SIZE = 4096

## Authentication - Generate Bearer

For details about getting the `bearer_token`, please see the `utils.py` file.

In [17]:
bearer_token = u.get_bearer_token(CLIENT_ID, USERNAME, PASSWORD, AUTH_URL)
if bearer_token:
    display(Markdown('**Authentication Successful**: Bearer token OK'))
else:
    display(Markdown('**Authentication Failed**: Cannot obtain the Bearer token'))

**Authentication Successful**: Bearer token OK

In [18]:
req_headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json"
}

## Factiva Retrieval API Query

In [19]:
frapi_query = {
  "data": {
    "attributes": {
      "response_limit": 3,
      "query": {
        "search_filters": [
          {
            "scope": "Language",
            "value": "en"
          }
        ],
        # "What are NASA's planned missions to the Moon in 2025, and what are their primary objectives?"
        # "Summarise the latest earnings report from Microsoft Corp"
        "value": "Summarise the latest earnings report from Microsoft Corp"
      }
    },
    "id": "GenAIRetrievalExample",
    "type": "genai-content"
  }
}


## Send Query and Receive Chunks from the Retrieval API

In [21]:
chunks_resp = r.post(f"https://{API_HOST}/content/gen-ai/retrieve", json=frapi_query, headers=req_headers)

if chunks_resp.status_code == 200:
    print('Successfully retrieved chunks')
else:
    print(f"Request Failed: {chunks_resp.json()}")

Successfully retrieved chunks


## Print Chunks

In [22]:
chunks_obj = chunks_resp.json()
for article in chunks_obj['data']:
    display(Markdown(f"### {article['attributes']['headline']['main']['text']}"))
    display(Markdown(f"**{article['meta']['source']['name']}** - {article['attributes']['publication_date']} - {article['meta']['original_doc_id']} - Lang: {article['meta']['language']['code']}"))
    display(Markdown(f"{article['attributes']['snippet']['content'][0]['text']} {article['attributes']['content'][0]['text']}"))
    display(Markdown(f"---"))

### Investigating Microsoft's Standing In Software Industry Compared To Competitors

**Benzinga.com** - 2025-02-13 - BNZNGA0020250213el2d000xg - Lang: en

In today's rapidly changing and fiercely competitive business landscape, it is essential for investors and industry enthusiasts to thoroughly analyze companies. In this article, we will conduct a comprehensive industry comparison, ... Investigating Microsoft's Standing In Software Industry Compared To Competitors In today's rapidly changing and fiercely competitive business landscape, it is essential for investors and industry enthusiasts to thoroughly analyze companies. In this article, we will conduct a comprehensive industry comparison, evaluating Microsoft (NASDAQ:MSFT) against its key competitors in the Software industry. By examining key financial metrics, market position, and growth prospects, we aim to provide valuable insights for investors and shed light on company's performance within the industry. Microsoft Background Microsoft develops and licenses consumer and enterprise software. It is known for its Windows operating systems and Office productivity suite. The company is organized into three equally sized broad segments: productivity and business processes (legacy Microsoft Office, cloud-based Office 365, Exchange, SharePoint, Skype, LinkedIn, Dynamics), intelligence cloud (infrastructure- and platform-as-a-service offerings Azure, Windows Server OS, SQL Server), and more personal computing (Windows Client, Xbox, Bing search, display advertising, and Surface laptops, tablets, and desktops). Company P/E P/B P/S ROE EBITDA (in billions) Gross Profit (in billions) Revenue Growth Microsoft Corp 32.96 10.05 11.67 8.17% $36.79 $47.83 12.27% Oracle Corp 42.11 35.04 8.91 25.66% $5.75 $9.97 8.64% ServiceNow Inc 143.63 21.11 18.64 4.06% $0.62 $2.33 21.34% Palo Alto Networks Inc 51.03 21.84 16.86 6.33% $0.45 $1.58 13.88% CrowdStrike Holdings Inc 852.22 35.01 29.06 -0.57% $0.05 $0.76 28.52% Fortinet Inc 47.65 55.26 13.96 90.26% $0.66 $1.24 13.0% Gen Digital Inc 26.89 7.86 4.41 7.48% $0.45 $0.79 4.01% Monday.Com Ltd 512.47 15.38 17.14 -1.28% $-0.02 $0.23 32.67% Dolby Laboratories Inc 30.73 3.19 6.14 2.72% $0.11 $0.32 13.13% CommVault Systems Inc 46.99 27.29 8.55 3.9% $0.02 $0.21 21.13% QXO Inc 23.51 1.21 21.42 -0.21% $-0.03 $0.01 -2.0% Qualys Inc 29.69 10.59 8.49 10.53% $0.05 $0.13 8.36% SolarWinds Corp 83.23 2.29 4 0.94% $0.07 $0.18 5.5% Progress Software Corp 37.07 5.64 3.37 0.27% $0.05 $0.18 21.47% Teradata Corp 21.22 17.72 1.38 32.0% $0.08 $0.27 0.46% Average 139.17 18.53 11.6 13.01% $0.59 $1.3 13.58% Upon closer analysis of Microsoft, the following trends become apparent: * A Price to Earnings ratio of 32.96 significantly below the industry average by 0.24x suggests undervaluation. This can make the stock appealing for those seeking growth. * Considering a Price to Book ratio of 10.05, which is well below the industry average by 0.54x, the stock may be undervalued based on its book value compared to its peers. * The Price to Sales ratio of 11.67, which is 1.01x the industry average, suggests the stock could potentially be overvalued in relation to its sales performance compared to its peers. * The Return on Equity (ROE) of 8.17% is 4.84% below the industry average, suggesting potential inefficiency in utilizing equity to generate profits. * Compared to its industry, the company has higher Earnings Before Interest, Taxes, Depreciation, and Amortization (EBITDA) of $36.79 Billion, which is 62.36x above the industry average, indicating stronger profitability and robust cash flow generation. * With higher gross profit of $47.83 Billion, which indicates 36.79x above the industry average, the company demonstrates stronger profitability and higher earnings from its core operations. * The company is witnessing a substantial decline in revenue growth, with a rate of 12.27% compared to the industry average of 13.58%, which indicates a challenging sales environment. Debt To Equity Ratio The debt-to-equity (D/E) ratio indicates the proportion of debt and equity used by a company to finance its assets and operations. Considering the debt-to-equity ratio in industry comparisons allows for a concise evaluation of a company's financial health and risk profile, aiding in informed decision-making. By analyzing Microsoft in relation to its top 4 peers based on the Debt-to-Equity ratio, the following insights can be derived: * Among its top 4 peers, Microsoft has a stronger financial position with a lower debt-to-equity ratio of 0.21. * This indicates that the company relies less on debt financing and maintains a more favorable balance between debt and equity, which can be viewed positively by investors. Key Takeaways For Microsoft in the Software industry, the PE and PB ratios suggest that the company is undervalued compared to its peers. However, the high PS ratio indicates that the market values Microsoft's sales more highly. In terms of ROE, EBITDA, and gross profit, Microsoft outperforms its peers, reflecting strong profitability and operational efficiency. The low revenue growth rate may indicate a need for Microsoft to focus on expanding its top line to align with industry trends. This article was generated by Benzinga's automated content engine and reviewed by an editor. © 2025 Benzinga.com. Benzinga does not provide investment advice. All rights reserved.

---

### This Magnificent Seven Tech Hits New Low In 2025; Is Microsoft Stock A Buy In February?

...

---

###                            Microsoft Stock: A Deep Dive Into Analyst Perspectives (18 Ratings)

**Benzinga.com** - 2024-12-18 - BNZNGA0020241220ekci0005s - Lang: en

In the preceding three months, 18 analysts have released ratings for Microsoft (NASDAQ:MSFT), presenting a wide array of perspectives from bullish to bearish. insights into expectations for the relative performance of Microsoft compared to the broader market. * Price Targets: Gaining insights, analysts provide estimates for the future value of Microsoft's stock. This comparison reveals trends in analysts' expectations over time. Understanding these analyst evaluations alongside key financial indicators can offer valuable insights into Microsoft's market standing. Stay informed and make well-considered decisions with our Ratings Table. Stay up to date on Microsoft analyst ratings. Delving into Microsoft's Background Microsoft develops and licenses consumer and enterprise software. It is known for its Windows operating systems and Office productivity suite. The company is organized into three equally sized broad segments: productivity and business processes (legacy Microsoft Office, cloud-based Office 365, Exchange, SharePoint, Skype, LinkedIn, Dynamics), intelligence cloud (infrastructure- and platform-as-a-service offerings Azure, Windows Server OS, SQL Server), and more personal computing (Windows Client, Xbox, Bing search, display advertising, and Surface laptops, tablets, and desktops). Microsoft's Financial Performance Market Capitalization Analysis: The company's market capitalization is above the industry average, indicating that it is relatively larger in size compared to peers. This may suggest a higher level of investor confidence and market recognition. Revenue Growth: Over the 3 months period, Microsoft showcased positive performance, achieving a revenue growth rate of 16.04% as of 30 September, 2024. This reflects a substantial increase in the company's top-line earnings. As compared to competitors, the company surpassed expectations with a growth rate higher than the average among peers in the Information Technology sector. Net Margin: Microsoft's financial strength is reflected in its exceptional net margin, which exceeds industry averages. With a remarkable net margin of 37.61%, the company showcases strong profitability and effective cost management. Return on Equity (ROE): Microsoft's ROE excels beyond industry benchmarks, reaching 8.87%. This signifies robust financial management and efficient use of shareholder equity capital. Return on Assets (ROA): The company's ROA is a standout performer, exceeding industry averages. With an impressive ROA of 4.77%, the company showcases effective utilization of assets. Debt Management: Microsoft's debt-to-equity ratio is below the industry average at 0.21, reflecting a lower dependency on debt financing and a more conservative financial approach. The Core of Analyst Ratings: What Every Investor Should Know Analysts are specialists within banking and financial systems that typically report for specific stocks or within defined sectors. These people research company financial statements, sit in conference calls and meetings, and speak with relevant insiders to determine what are known as analyst ratings for stocks. Typically, analysts will rate each stock once a quarter. Some analysts will also offer forecasts for metrics like growth estimates, earnings, and revenue to provide further guidance on stocks. Investors who use analyst ratings should note that this specialized advice comes from humans and may be subject to error. Which Stocks Are Analysts Recommending Now? Benzinga Edge gives you instant access to all major analyst upgrades, downgrades, and price targets. Sort by accuracy, upside potential, and more. Click here to stay ahead of the market. This article was generated by Benzinga's automated content engine and reviewed by an editor. Latest Ratings for MSFT Date Firm Action From To Feb 2022 Tigress Financial Maintains Buy Jan 2022 Citigroup Maintains Buy Jan 2022 Morgan Stanley Maintains Overweight View More Analyst Ratings for MSFT View the Latest Analyst Ratings © 2024 Benzinga.com. Benzinga does not provide investment advice. All rights reserved.

---

## Retrieval API Conclusion

Up to this point it the seen functionalities are connected with the Retrieval API.

However, given that this service is an intermediate component in a full-stack solution, the below Test LLM steps and the [Read Article](2_read_article.ipynb) notebook are the two complementary avenues to get a full working solution.

## Test LLM

This is a downstream step that is only illustrative on how the response generation stage can be implemented. In the below example only a few articles are used to repond the prompt in a locally running LLM. The passed request is built from the original prompt enhanced with a few retrieved articles as additional context.

The goal of using only a few articles is to improve performance as the LLM can take several minutes to summarise all returned articles given the natural limitations of a desktop hardware.

The tested LLM was Microsoft's Phi-4 with 14Bn parameters. Response generations took between 1 and 2 minutes for a context between 3K to 4K tokens.

In [11]:
prompt_text = f"{frapi_query['data']['attributes']['query']['value']}. To answer use the following information:\n\n"

chunks_obj = chunks_resp.json()['data']
process_chunks = chunks_obj[0:3]
for article in process_chunks:
    prompt_text += f"{article['attributes']['headline']['main']['text']} - {article['attributes']['publication_date']}"
    prompt_text += f"{article['attributes']['snippet']['content'][0]['text']} {article['attributes']['content'][0]['text']}\n------\n"


In [12]:
print(prompt_text)

Summarise the latest earnings report from Apple Inc. To answer use the following information:

InvestorNewsBreaks – Apple Inc. (NASDAQ: AAPL) Eyes Future Growth Beyond Q1 Results - 2025-02-10Apple (NASDAQ: AAPL) reported record revenue of $124.3 billion in Q1 2025, driven by a 14% increase in services revenue, which helped offset a 0.8% decline in iPhone sales attributed to a weaker Chinese market. Looking ahead, the company ... InvestorNewsBreaks – Apple Inc. (NASDAQ: AAPL) Eyes Future Growth Beyond Q1 Results Apple (NASDAQ: AAPL) reported record revenue of $124.3 billion in Q1 2025, driven by a 14% increase in services revenue, which helped offset a 0.8% decline in iPhone sales attributed to a weaker Chinese market. Looking ahead, the company plans to launch the iPhone SE 4, explore a potential flip phone, and develop advanced wearable technologies. Apple’s strategic focus on AI integration and expansion into emerging markets is expected to drive future growth. As of 8:26 a.m. EST on

This is a quick check to ensure the estimated number of Tokes from the `prompt_text` are below the `LLM_CONTEXT_SIZE`.

In [13]:
len(re.split(r"\b\w+\b", prompt_text)) * 1.25 < LLM_CONTEXT_SIZE

True

LLM Payload requesting a structured response

In [24]:
prompt_payload = {
    "model": "phi-4",
    "messages": [
      {
        "role": "system",
        "content": "You are an expert news analyst that respond in a professional manner. Use only the context provided by the user to answer its request at the beginning of the prompt."
      },
      {
        "role": "user",
        "content": prompt_text
      }
    ],
    "response_format": {
      "type": "json_schema",
      "json_schema": {
        "name": "prompt_response",
        "strict": "true",
        "schema": {
          "type": "object",
          "properties": {
            "response": {
              "type": "string"
            }
          },
        "required": ["response"]
        }
      }
    },
    "temperature": 0.1,
    "max_tokens": 500,
    "stream": False
  }

The endpoint URL will depend on where the model is running. A tool like LM Studio provides details for developers on the ip address that can be used for this API request. Nonetheless, the general structure is simialar to the OpenAI API and can easily be adapted to use their service.

This is the request that can take 1+ minutes to complete.

In [15]:
summ_resp = r.post(f"http://192.168.1.10:1234/v1/chat/completions", json=prompt_payload, headers={'Content-Type': 'application/json'})

In [16]:
display(HTML(f"<h3>Response:</h3><p>{summ_resp.json()['choices'][0]['message']['content']}</p>"))